<a href="https://colab.research.google.com/github/AkhdanFirdaus/bmn-model/blob/main/final_ta_klasifikasi_teks_bert_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasifikasi Teks Laporan Mobil Dinas dengan BERT

In [39]:
import pandas as pd
import numpy as np

### Load Data

In [ ]:
# Mount Drive

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loc_master = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_master_kendaraan.csv'
loc_invoice = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_invoice.csv'
loc_sukucadang = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_sukucadang.csv'
loc_kerusakan = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_kategori_kerusakan.csv'
loc_laporan = '/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Raw Data/raw_laporan.csv'

data_master_raw = pd.read_csv(loc_master)
data_invoice_raw = pd.read_csv(loc_invoice)
data_sukucadang_raw = pd.read_csv(loc_sukucadang)
data_kerusakan_raw = pd.read_csv(loc_kerusakan)
data_laporan_raw = pd.read_csv(loc_laporan)

# 4.2 Pemilahan Data

## 4.2.1 Perbaikan Data

In [ ]:
data_invoice_fixed = data_invoice_raw.copy()

# Mengisi seluruh data NaN yang sejenis dengan mengisi sesuai karakeristiknya
data_invoice_fixed['No'].fillna(method='ffill', inplace=True)
data_invoice_fixed['Tanggal'].fillna(method='ffill', inplace=True)
data_invoice_fixed['Nama'].fillna(method='ffill', inplace=True)
data_invoice_fixed['Harga Satuan'] = data_invoice_fixed['Harga Satuan'].str.replace(',', '').astype(int)
data_invoice_fixed['Total'] = data_invoice_fixed['Total'].str.replace(',', '').astype(int)

# Mengganti Nama Atribut
data_invoice_fixed = data_invoice_fixed.rename(columns={
    'No': 'no',
    'Tanggal': 'tanggal',
    'Nama': 'nama',
    'Kategori': 'kategori',
    'Uraian Pekerjaan': 'pekerjaan',
    'Qty': 'qty',
    'Harga Satuan': 'harga',
    'Total': 'total',
    'Subtotal': 'subtotal',
})

# Group and sum
data_invoice_fixed = data_invoice_fixed.groupby(['nama', 'kategori', 'pekerjaan']).agg({
    'qty': 'sum',
    'total': 'sum',
    'harga': lambda x: round(x.mean())
}).reset_index()

# Ditujukan untuk visualisasi
data_invoice_per_mobil = data_invoice_fixed.groupby(['nama']).agg({'total': 'sum'}).reset_index()
data_invoice_per_kategori = data_invoice_fixed.groupby('kategori')['total'].mean().apply(lambda x: round(x)).reset_index()

In [ ]:
data_sukucadang_fixed = data_sukucadang_raw.copy()

# Mengganti Nama Atribut
data_sukucadang_fixed = data_sukucadang_fixed.rename(columns={
    'Komponen': 'pekerjaan',
    'Keterangan' : 'keterangan_pekerjaan'
})

In [ ]:
data_kerusakan_fixed = data_kerusakan_raw.copy()

# Menggabungkan data kategori dan data keterangan untuk menambah konteks
data_kerusakan_fixed['Keterangan'] =  data_kerusakan_fixed['Keterangan'] + ', ' + data_kerusakan_fixed['Kategori']

# Mengganti Nama Atribut
data_kerusakan_fixed = data_kerusakan_fixed.rename(columns={
    'Kerusakan': 'kerusakan',
    'Keterangan': 'keterangan_kerusakan',
    'Suku Cadang': 'pekerjaan',
    'Kategori':'kategori'
})

In [ ]:
data_laporan_fixed = data_laporan_raw.rename(columns={
    'Laporan': 'laporan',
    'Masalah': 'masalah'
})

In [ ]:
data_invoice_fixed.head()

,nama,kategori,pekerjaan,qty,total,harga
0,Ford Everest B 1700 PQR,Bahan Bakar,Tune Up Purging Diesel,2,1150000,575000
1,Ford Everest B 1700 PQR,Bumper,Perbaikan Bumper Depan,1,250000,250000
2,Ford Everest B 1700 PQR,Clean,Uccu Cleanner,2,150000,75000
3,Ford Everest B 1700 PQR,Filter Bahan Bakar,Filter Solar Ford Everest,1,175000,175000
4,Ford Everest B 1700 PQR,Filter Oli,Filter Oli Ford TB001,1,150000,150000


In [ ]:
len(data_sukucadang_fixed.drop_duplicates(subset=['pekerjaan']))

205

In [ ]:
data_kerusakan_fixed.head()

,kerusakan,keterangan_kerusakan,kategori
0,Masalah Sistem Knalpot,Sistem pembuangan bertanggung jawab untuk meng...,"Filter Udara,Filter Oli,Oli,Bahan Bakar,Cleane..."
1,Masalah Transmisi,Masalah transmisi kendaraan roda empat adalah ...,"Transmisi,Pemasangan,Kopling,Joint,Bearing,Pem..."
2,Masalah Suspensi,Suspensi kendaraan roda empat adalah kumpulan ...,"Shockbreaker,Spooring,Balancing,Hidrolik,Pemas..."
3,Gangguan Listrik,Gangguan listrik pada kendaraan roda empat ter...,"Lampu,Listrik,Pemasangan"
4,Masalah Sistem Bahan Bakar,Masalah sistem bahan bakar kendaraan roda empa...,"Filter Bahan Bakar,Bahan Bakar,Cleaner,Pemasan..."


In [ ]:
data_laporan_fixed.head()

,laporan,masalah
0,Saya melihat indikator check engine menyala di...,Masalah Mesin
1,"Saat saya menginjak rem, terdengar suara berde...",Kegagalan Rem
2,Saya merasakan getaran yang tidak normal saat ...,"Masalah Kemudi, Masalah Ban"
3,Tombol power window di pintu pengemudi tidak b...,Gangguan Listrik
4,Ada bau tidak sedap yang muncul dari ventilasi...,Masalah Sistem Pendingin


# 4.3 Data Preparation

## 4.3.1 Pemilahan Data

### Perbaikan Data

## 4.3.2 Pembersihan Data

In [93]:
sample_text = 'Mesin kendaraan terdengar berisik dan tidak stabil saat digunakan.'

### Casefolding

In [94]:
def casefolding(val):
  return str(val).lower()

In [95]:
sample_text = casefolding(sample_text)
print(sample_text)

mesin kendaraan terdengar berisik dan tidak stabil saat digunakan.


### Cleaning

In [96]:
import re

def clearformat(val):
  # Membersihkan Whitespace
  val = re.sub(r'\s+', ' ', val)

  # Hanya Mengambil karakter alfanumerik
  val = re.sub("[^a-zA-Z0-9]", " ", val)

  return val

In [97]:
sample_text = clearformat(sample_text)
print(sample_text)

mesin kendaraan terdengar berisik dan tidak stabil saat digunakan 


### Stemming

Hanya dilakuan pada
- data sukucadang = Keterangan Pekerjaan
- data kerusakan = Keterangan Kerusakan
- data level = keterangan level



In [98]:
pip install Sastrawi

In [99]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer = StemmerFactory().create_stemmer()

def stemming(val):
  return stemmer.stem(str(val))

In [100]:
sample_text = stemming(sample_text)
print(sample_text)

mesin kendara dengar berisik dan tidak stabil saat guna


### Stopword Removal

Hanya dilakuan pada
- data sukucadang = Keterangan Pekerjaan
- data kerusakan = Keterangan Kerusakan
- data level = keterangan level

In [101]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stopword_remover = StopWordRemoverFactory().create_stop_word_remover()

def stopwordremove(val):
  return stopword_remover.remove(str(val))

In [102]:
sample_text = stopwordremove(sample_text)
print(sample_text)

mesin kendara dengar berisik tidak stabil guna


### Tokenisasi

In [107]:
!pip install transformers tensorflow

In [108]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [20]:
BERT_NAME = 'indobenchmark/indobert-lite-base-p1'
bert_tokenizer = BertTokenizer.from_pretrained(BERT_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.


In [109]:
def tokenisasi(tokenizer, max_len, sentences):
  input_ids, attention_mask = [], []
  for sentence in sentences:
    # proses preprocessing sebelumnya akan dimasukkan disini
    tokenized = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids.append(tokenized['input_ids'])
    attention_mask.append(tokenized['attention_mask'])

  return {
    'input_ids': tf.convert_to_tensor(np.asarray(input_ids).squeeze(), dtype=tf.int32),
    'attention_mask': tf.convert_to_tensor(np.asarray(attention_mask).squeeze(), dtype=tf.int32)
  }

In [116]:
hasil_tokenisasi = tokenisasi(bert_tokenizer, 256, [
  sample_text
])

print('shape = ', hasil_tokenisasi['input_ids'].shape, hasil_tokenisasi['attention_mask'].shape)
print('\ninput_ids = ', hasil_tokenisasi['input_ids'][:20])
print('attention_mask = ', hasil_tokenisasi['attention_mask'][:20])
print('\ntoken = ', bert_tokenizer.convert_ids_to_tokens(hasil_tokenisasi['input_ids'][:20]))

shape =  (256,) (256,)

input_ids =  tf.Tensor(
[    2  1315  1596    85  5595 20872   119  4860  3208     3     0     0
     0     0     0     0     0     0     0     0], shape=(20,), dtype=int32)
attention_mask =  tf.Tensor([1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0], shape=(20,), dtype=int32)

token =  ['[CLS]', 'mesin', 'kend', '##ara', 'dengar', 'berisik', 'tidak', 'stabil', 'guna', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


## 4.3.3 Konstruksi Data

### Penentuan Fitur

Akan dibuat sebuah dataset untuk mendeteksi konteks
1. menggabungkan data master dan invoice berdasar nomor polisi
2. menggabungkan data diatas dengan sukucadang berdasar pekerjaan

### Pemilihan Atribut

## 4.3.4 Pelabelan Data

### Penentuan Label

In [ ]:
# Ekstraksi label kerusakan dan level dari dataset
LABELS = data_kerusakan_construct['kerusakan'].tolist()
print("Label kerusakan = ", LABELS)

### Penerapan Label

In [ ]:
# Menambahkan Kolom Label
data_invoice_labeled = data_invoice_construct.copy()
data_invoice_labeled[LABELS] = 0
data_invoice_labeled.head()

In [ ]:
# Menambahkan Kolom Label
data_laporan_labeled = data_laporan_construct.copy()
data_laporan_labeled[LABELS] = 0
data_laporan_labeled.head()

In [ ]:
# Fungsi Labeling
def labeling(val, dataset, key):
  labels = {}
  konteks = ''
  for index, row in dataset.iterrows():
    match_condition = 1 if val in row[key].split(',') else 0
    konteks += ' ' + row['konteks'] if match_condition == 1 else ""
    labels.update({row['kerusakan']: match_condition})

  labels.update({'konteks': konteks})
  return labels

In [ ]:
for index, row in data_invoice_labeled.iterrows():
    update_value = labeling(row['kategori'], data_invoice_labeled, 'kategori')
    data_invoice_labeled.loc[index, list(update_value.keys())] = list(update_value.values())

In [ ]:
for index, row in data_laporan_labeled.iterrows():
    update_value = labeling(row['masalah'], data_laporan_labeled, 'masalah')
    data_laporan_labeled.loc[index, list(update_value.keys())] = list(update_value.values())

## 4.3.5 Integrasi Data

### Penggabungan Data

### Pembentukan Dataset Akhir

### Balancing Data

In [ ]:
def balance_dataset(dataset):
    balanced = pd.DataFrame(columns=dataset.columns)
    for label in dataset.columns[1:]:
        temp = dataset[dataset[label] == 1]
        temp = temp.sample(n=min(15, len(temp)), random_state=42)
        balanced = pd.concat([balanced, temp])
    return balanced

# This is the function to preprocess the dataset
balanced = balance_dataset(newdataset2)
balanced.shape

### Split Dataset

# 4.4 Modeling

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Kuliah/TA Akhdan Musyaffa Firdaus/Bimbingan - Akhdan Musyaffa Firdaus/Dataset/Dipilah/final_dataset3.csv', sep='\t')
LABELS = dataset.columns[1:]

In [ ]:
!pip install transformers tensorflow Sastrawi

In [ ]:
import tensorflow as tf
# tf.config.run_functions_eagerly(True)
from transformers import BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split

## Hyperparameters

In [ ]:
MAX_LEN=256
TRAIN_BATCH_SIZE=32
TEST_BATCH_SIZE=32
EPOCHS=10
LEARNING_RATE=2e-05
TRAIN_SPLIT=0.8
TEST_SPLIT=0.2

In [ ]:
# Prepare Pre-trained Model dan Tokenizer
BERT_NAME = 'indobenchmark/indobert-lite-base-p1'
bert_tokenizer = BertTokenizer.from_pretrained(BERT_NAME)
bert_model = TFBertModel.from_pretrained(BERT_NAME)

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmer = StemmerFactory().create_stemmer()
stopword_remover = StopWordRemoverFactory().create_stop_word_remover()

def casefolding(val):
  return str(val).lower()

def stemming(val):
  return stemmer.stem(str(val))

def stopwordremove(val):
  return stopword_remover.remove(str(val))

def preprocess_sentences(tokenizer, max_len, sentences):
  input_ids, attention_mask = [], []
  for sentence in sentences:
    input = casefolding(sentence)
    input = stemming(input)
    input = stopwordremove(input)
    tokenized = tokenizer.encode_plus(
        input,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids.append(tokenized['input_ids'])
    attention_mask.append(tokenized['attention_mask'])

  return {
    'input_ids': tf.convert_to_tensor(np.asarray(input_ids).squeeze(), dtype=tf.int32),
    'attention_mask': tf.convert_to_tensor(np.asarray(attention_mask).squeeze(), dtype=tf.int32)
  }

In [ ]:
# Percobaan penggunaan tokenisasi
tokenized = preprocess_sentences(bert_tokenizer, MAX_LEN, [
    'Mobil saya tidak bisa di starter',
    'AC tidak dingin'
])

print(tokenized)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
  dataset['konteks'],
  dataset[LABELS],
  test_size=TEST_SPLIT,
  shuffle=True,
  random_state=42
)

X_train = preprocess_sentences(bert_tokenizer, MAX_LEN, X_train)
X_test = preprocess_sentences(bert_tokenizer, MAX_LEN, X_test)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(TRAIN_BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(TEST_BATCH_SIZE)

In [ ]:
def classification_model(bert_encoder, num_labels, max_len, learning_rate):
  input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='input_ids')
  attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name='attention_mask')

  encoding_layer = bert_encoder(input_ids, attention_mask)[0]

  l = tf.keras.layers.GlobalAveragePooling1D(name='pooling_layer')(encoding_layer)
  l = tf.keras.layers.Dropout(0.1, name='dropout_layer')(l)
  l = tf.keras.layers.Dense(num_labels, activation='sigmoid', name='output_layer')(l)

  model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[l])

  OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  LOSS = tf.keras.losses.BinaryCrossentropy()
  METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]

  model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

  return model

model = classification_model(
    bert_encoder=bert_model,
    num_labels=len(LABELS),
    max_len=MAX_LEN,
    learning_rate=LEARNING_RATE,
)

In [ ]:
class CustomDataset(tf.keras.utils.Sequence):
  def __init__(self, data, tokenizer, max_len):
    self.data = data.reset_index(drop=True)
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.indexes = np.arrange(len(self.data))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    row = self.data.iloc[index]

    konteks = row['konteks'].tolist()
    sparepart = row['sparepart'].tolist()
    price = row['price'].tolist()
    labels = row.drop(['konteks', 'sparepart', 'price'], axis=1)

    input = casefolding(konteks)
    input = stemming(input)
    input = stopwordremove(input)

    tokenized = self.tokenizer.encode_plus(
        input,
        add_special_tokens=True,
        max_length=self.max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='tf'
    )

    input_ids = tf.convert_to_tensor(np.asarray(tokenized['input_ids']).squeeze(), dtype=tf.int32)
    attention_mask = tf.convert_to_tensor(np.asarray(tokenized['attention_mask']).squeeze(), dtype=tf.int32)

    return {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'labels': labels,
      'sparepart': sparepart,
      'price': price
    }


In [ ]:
class ClassificationModelWithRecommendation(tf.keras.Model):
  def __init__(self, bert_encoder, num_labels, num_spare_parts, max_len, learning_rate):
    super(ClassificationModelWithRecommendation, self).__init__()
    self.bert_encoder = bert_encoder
    self.num_labels = num_labels
    self.num_spare_parts = num_spare_parts
    self.max_len = max_len
    self.learning_rate = learning_rate

    # Init classification submodel
    self.classification_model = self.build_classification_model()

    # Init spareparts prediction submodel
    self.spare_part_model = self.build_spare_part_model()

    # Init estimated price submodel
    self.price_estimation_model = self.build_price_estimation_model()

  def build_classification_model(self):
    input_ids = tf.keras.layers.Input(shape=(self.max_len,), dtype=tf.int32, name='input_ids')
    attention_mask = tf.keras.layers.Input(shape=(self.max_len,), dtype=tf.int32, name='attention_mask')

    encoding_layer = self.bert_encoder(input_ids, attention_mask)[0]

    l = tf.keras.layers.GlobalAveragePooling1D(name='poolling_layer')(encoding_layer)
    l = tf.keras.layers.Dropout(0.1, name='dropout_layer')(l)
    l = tf.keras.layers.Dense(self.num_labels, action='sigmoid', name='output_layer')(l)

    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[l])

    OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
    LOSS = tf.keras.losses.BinaryCrossentropy()
    METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy')]

    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

    return model

  def build_spare_part_model(self):
    spare_part_input = tf.keras.layers.Input(shape=(self.num_labels,), name='spare_part_input')
    spare_part_output = tf.keras.layers.Dense(self.num_spare_parts, activation='softmax', name='spare_part_output')(spare_part_input)
    model = tf.keras.Model(inputs=spare_part_input, outputs=spare_part_output)
    return model

  def build_price_estimation_model(self):
    price_input = tf.keras.layers.Input(shape=(self.num_labels,), name='price_input')
    price_output = tf.keras.layers.Dense(self.num_spare_parts, activation='linear', name='price_output')(price_input)
    model = tf.keras.Model(inputs=price_input, outputs=price_output)
    return model

  def call(self, inputs):
    classification_output = self.classification_model(inputs)
    spare_part_probs = self.spare_part_model(classification_output)
    price_estimations = self.price_estimation_model(classification_output)
    return classification_output, spare_part_probs, price_estimations


model_with_recommendation = ClassificationModelWithRecommendation(
    bert_encoder=bert_model,
    num_labels=len(LABELS),
    num_spare_parts=len(),
    max_len=MAX_LEN,
    learning_rate=LEARNING_RATE
)

TypeError: ignored

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109081344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [ ]:
model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=test_dataset,
)

Epoch 1/10


8/8 [==============================] - 74s 2s/step - loss: 0.5048 - accuracy: 0.8147 - val_loss: 0.3267 - val_accuracy: 0.8968
Epoch 2/10
8/8 [==============================] - 12s 2s/step - loss: 0.3396 - accuracy: 0.8814 - val_loss: 0.2826 - val_accuracy: 0.8906
Epoch 3/10
8/8 [==============================] - 12s 2s/step - loss: 0.2812 - accuracy: 0.8892 - val_loss: 0.2567 - val_accuracy: 0.9045
Epoch 4/10
8/8 [==============================] - 12s 2s/step - loss: 0.2294 - accuracy: 0.9032 - val_loss: 0.2111 - val_accuracy: 0.9384
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.1862 - accuracy: 0.9263 - val_loss: 0.1935 - val_accuracy: 0.9353
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 0.1512 - accuracy: 0.9504 - val_loss: 0.1617 - val_accuracy: 0.9553
Epoch 7/10
8/8 [==============================] - 13s 2s/step - loss: 0.1283 - accuracy: 0.9493 - val_loss: 0.1368 - val_accuracy: 0.9538
Epoch 8/10
8/8 [=============================

# 4.5 Model Evaluation

In [ ]:
model.evaluate(test_dataset)

2/2 [==============================] - 1s 504ms/step - loss: 0.1187 - accuracy: 0.9646


[0.11871889233589172, 0.964560866355896]

In [ ]:
inputs = ['AC mobil tidak dingin', 'Rem pada mobil tidak bekerja', 'Kaca spion rusak harus diganti']
tokenized = preprocess_sentences(bert_tokenizer, MAX_LEN, inputs)
predictions = model.predict(tokenized)

# Define the threshold
threshold = 0.5

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, 0)

# Print the binary predictions
print(LABELS)
print(binary_predictions)

1/1 [==============================] - 7s 7s/step
Index(['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi',
       'gangguan listrik', 'masalah sistem bahan bakar', 'masalah kemudi',
       'masalah ban', 'masalah mesin', 'kegagalan rem',
       'masalah sistem pendingin', 'kerusakan aksesoris interior eksterior'],
      dtype='object')
[[0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have predictions and ground truth labels
predictions = model.predict(test_dataset)  # Get model predictions
threshold = 0.5  # Set threshold for label prediction

# Convert probability predictions to binary predictions
binary_predictions = np.where(predictions >= threshold, 1, 0)

# Compute precision, recall, and F1-score for each label
label_precisions = precision_score(y_test, binary_predictions, average=None)
label_recalls = recall_score(y_test, binary_predictions, average=None)
label_f1_scores = f1_score(y_test, binary_predictions, average=None)

# Print accuracy metrics for each label
for label, precision, recall, f1_score in zip(LABELS, label_precisions, label_recalls, label_f1_scores):
    print(f"Label: {label}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
    print()


2/2 [==============================] - 5s 468ms/step
Label: masalah sistem knalpot
Precision: 1.0
Recall: 0.8333333333333334
F1-score: 0.9090909090909091

Label: masalah transmisi
Precision: 0.8888888888888888
Recall: 1.0
F1-score: 0.9411764705882353

Label: masalah suspensi
Precision: 0.3333333333333333
Recall: 0.3333333333333333
F1-score: 0.3333333333333333

Label: gangguan listrik
Precision: 0.875
Recall: 1.0
F1-score: 0.9333333333333333

Label: masalah sistem bahan bakar
Precision: 1.0
Recall: 0.6666666666666666
F1-score: 0.8

Label: masalah kemudi
Precision: 0.7142857142857143
Recall: 1.0
F1-score: 0.8333333333333333

Label: masalah ban
Precision: 0.75
Recall: 0.8571428571428571
F1-score: 0.7999999999999999

Label: masalah mesin
Precision: 1.0
Recall: 0.3333333333333333
F1-score: 0.5

Label: kegagalan rem
Precision: 1.0
Recall: 1.0
F1-score: 1.0

Label: masalah sistem pendingin
Precision: 0.8
Recall: 1.0
F1-score: 0.888888888888889

Label: kerusakan aksesoris interior eksterior
Pr

# 4.6 Deployment

In [ ]:
!rm model_output/klasifikasi.h5
model.save('model_output/klasifikasi.h5')

rm: cannot remove 'model_output/klasifikasi2.h5': No such file or directory


In [ ]:
loaded_model = tf.keras.models.load_model('/content/model_output/klasifikasi.h5', custom_objects={'TFBertModel': TFBertModel})

In [ ]:
inputs = ['mesin tidak bisa menyala', 'AC mobil tidak dingin', 'Mesin bersuara kasar', 'ban bocor dan harus diganti dengan yang baru', 'asap knalpot berwarna putih', 'kaca spion pecah dan harus diganti', 'lampu sein kendaraan tidak berfungsi']
tokenized = preprocess_sentences(bert_tokenizer, MAX_LEN, inputs)
predictions = loaded_model.predict(tokenized)

# Define the threshold
threshold = 0.5

# Apply the threshold to the predictions
binary_predictions = np.where(predictions > threshold, 1, 0)

# Print the binary predictions
print(LABELS)
print(binary_predictions)

1/1 [==============================] - 6s 6s/step
Index(['masalah sistem knalpot', 'masalah transmisi', 'masalah suspensi',
       'gangguan listrik', 'masalah sistem bahan bakar', 'masalah kemudi',
       'masalah ban', 'masalah mesin', 'kegagalan rem',
       'masalah sistem pendingin', 'kerusakan aksesoris interior eksterior'],
      dtype='object')
[[0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]]


## Implementasi Rumus menentukan tingkat kerusakan

In [ ]:
LABEL_WEIGHTS = dict.fromkeys(label_kerusakan, 0)

def measure_severity(val):
  treshold = 0.5

  NUMBER_LABEL_DETECTED = np.sum(val > treshold)
  LABEL_RESULTS = {}

  for label, count in zip(LABEL_WEIGHTS.keys(), val):
    if count > treshold:
      LABEL_RESULTS[label] = count

  category_weights = [LABEL_WEIGHTS[label] * count for label, count in LABEL_RESULTS.items()]
  severity_score = sum(category_weights) / NUMBER_LABEL_DETECTED

  return severity_score